In [76]:
import pandas as pd
import json
import re

In [123]:
input_file_path = '/Users/vik/Desktop/cs224nproject/Train_with_gpt_output_full.csv'
df = pd.read_csv(input_file_path, header=0)
df.head()

,author,text,timestamp,subreddit,language,gpt_output
0,randomcomment_FYI,首先向你的勇气致敬 现实情况的话 一方面小公司多试试 或者看看有没有小生意能上手做做的 至于...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""小公司"",\n ..."
1,Particular-Fig4183,如果你有正式的学历 理工类那种 润出外面还是能混得不错的 尤其是“反对清零以及伟大领袖的连任...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""洋人"",\n ""..."
2,Zealousideal-Wave822,潤去美國申請政庇鐵定過？ 首先你要被各種移民中介吃一遍！ 然後各種移民律師「基本很多華人律師...,2024-04-22,China_irl,zh,"[\n {\n ""entity"": ""潤"",\n ""e..."
3,BeechburgKrab,楼主大概是名校毕业，程序员，一路大厂实习工作这么过来？如果设身处地的想我会怎么做？首先国内这...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""楼主"",\n ""..."
4,matthiasxy,(从原英文留言翻译过来，这个社区不支持英文留言) 很抱歉打扰您！ 我不知道如何帮助您！ 或者...,2024-04-22,China_irl,en,"[\n {\n ""entity"": ""HR"",\n ""..."


In [107]:
# Function to create labels for the text
def create_labels_for_text(text, entities):
    words = list(text)  # To handle Chinese characters, we split by character
    word_labels = ["O"] * len(words)
    
    for entity in entities:
        entity_text = entity["entity"]
        entity_type = entity["entity_type"]
        sentiment = entity["sentiment"]
        
        entity_label = f"B-{entity_type}-{sentiment}"
        entity_chars = list(entity_text)
        entity_len = len(entity_chars)
        
        for i in range(len(words) - entity_len + 1):
            # Check if the sequence of characters matches the entity
            if words[i:i + entity_len] == entity_chars:
                word_labels[i] = entity_label
                for j in range(1, entity_len):
                    word_labels[i + j] = f"I-{entity_type}-{sentiment}"
                break
    
    return word_labels

def fix_json_format(json_str):
    # Remove newlines and extra spaces
    json_str = json_str.replace('\n', '').replace(' ', '')
    # Ensure every entity has the "entity" key
    json_str = re.sub(r'(\},\{)', r'\1"entity":', json_str)
    # Add quotes around keys if missing
    json_str = re.sub(r'([{,])(\s*?)(\w+)(:)', r'\1\2"\3"\4', json_str)
    # Add quotes around values if missing
    json_str = re.sub(r'(:)(\s*?)(\w+)([,}])', r'\1\2"\3"\4', json_str)
    return json_str


first_row_gpt_output = df.loc[0, 'gpt_output']
first_row_tex = df.loc[0, 'text']
cleaned_gpt_output1 = fix_json_format(first_row_gpt_output)
entities = json.loads(cleaned_gpt_output1)
generated_labels = create_labels_for_text(first_row_tex, entities)
print("Generated Labels:", generated_labels)

Generated Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG-neutral', 'I-ORG-neutral', 'I-ORG-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG-neutral', 'I-ORG-neutral', 'I-ORG-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC-neutral', 'I-LOC-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [124]:
import json
import re

# Function to create labels for the text
def create_labels_for_text(text, entities, is_chinese):
    if is_chinese:
        words = list(text)  # Split by character for Chinese
    else:
        words = text.split()  # Split by words for other languages
    
    word_labels = ["O"] * len(words)
    
    for entity in entities:
        entity_text = entity["entity"]
        entity_type = entity["entity_type"]
        sentiment = entity["sentiment"]
        
        entity_label = f"B-{entity_type}-{sentiment}"
        if is_chinese:
            entity_chars = list(entity_text)
        else:
            entity_chars = entity_text.split()
        entity_len = len(entity_chars)
        
        for i in range(len(words) - entity_len + 1):
            # Check if the sequence of words/characters matches the entity
            if words[i:i + entity_len] == entity_chars:
                word_labels[i] = entity_label
                for j in range(1, entity_len):
                    word_labels[i + j] = f"I-{entity_type}-{sentiment}"
                break
    
    return word_labels

# Function to clean and fix JSON string, ensuring "entity" key is present
def fix_json_format(json_str):
    # Remove newlines and extra spaces
    json_str = json_str.replace('\n', '').replace(' ', '')
    # Ensure every entity has the "entity" key
    json_str = re.sub(r'(\},\{)', r'\1"entity":', json_str)
    # Add quotes around keys if missing
    json_str = re.sub(r'([{,])(\s*?)(\w+)(:)', r'\1\2"\3"\4', json_str)
    # Add quotes around values if missing
    json_str = re.sub(r'(:)(\s*?)(\w+)([,}])', r'\1\2"\3"\4', json_str)
    return json_str

# Process each row in the dataframe
labels_list = []

for index, row in df.iterrows():
    try:
        gpt_output = row['gpt_output']
        text = row['text']
        language = row['language']
        subreddit = row['subreddit']
        
        # Determine if the text should be treated as Chinese
        is_chinese = (language == 'zh') or (subreddit == 'China_irl') or (subreddit == 'Doubangoosegroup') or (subreddit == 'real_china_irl') or (subreddit == 'liberalgoosegroup') or (subreddit == 'mohu') or (subreddit == 'hanren') or (subreddit == 'Taiwanese') or (subreddit == 'mohu') or (subreddit == 'hanren')
        cleaned_gpt_output = fix_json_format(gpt_output)
        entities = json.loads(cleaned_gpt_output)
        labels = create_labels_for_text(text, entities, is_chinese)
        labels_list.append(labels)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        labels_list.append([])  # Append an empty list or handle appropriately

# Add the processed labels to the dataframe
df['labels'] = labels_list

# Print the last 50 rows of the dataframe
print(df.tail(50))


Error processing row 121: Unterminated string starting at: line 1 column 2089 (char 2088)
Error processing row 257: Expecting ',' delimiter: line 1 column 76 (char 75)
Error processing row 277: Expecting value: line 1 column 2247 (char 2246)
Error processing row 335: Expecting ',' delimiter: line 1 column 78 (char 77)
Error processing row 420: Unterminated string starting at: line 1 column 2208 (char 2207)
Error processing row 426: Expecting ',' delimiter: line 1 column 2382 (char 2381)
Error processing row 452: Expecting ',' delimiter: line 1 column 88 (char 87)
Error processing row 467: Expecting ',' delimiter: line 1 column 79 (char 78)
Error processing row 580: Expecting ',' delimiter: line 1 column 80 (char 79)
Error processing row 585: Expecting ',' delimiter: line 1 column 79 (char 78)
Error processing row 662: Expecting ',' delimiter: line 1 column 82 (char 81)
Error processing row 686: Expecting ',' delimiter: line 1 column 76 (char 75)
Error processing row 803: Unterminated s

In [125]:
df.head(50)

,author,text,timestamp,subreddit,language,gpt_output,labels
0,randomcomment_FYI,首先向你的勇气致敬 现实情况的话 一方面小公司多试试 或者看看有没有小生意能上手做做的 至于...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""小公司"",\n ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Particular-Fig4183,如果你有正式的学历 理工类那种 润出外面还是能混得不错的 尤其是“反对清零以及伟大领袖的连任...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""洋人"",\n ""...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Zealousideal-Wave822,潤去美國申請政庇鐵定過？ 首先你要被各種移民中介吃一遍！ 然後各種移民律師「基本很多華人律師...,2024-04-22,China_irl,zh,"[\n {\n ""entity"": ""潤"",\n ""e...","[B-PER-neutral, O, B-LOC-neutral, I-LOC-neutra..."
3,BeechburgKrab,楼主大概是名校毕业，程序员，一路大厂实习工作这么过来？如果设身处地的想我会怎么做？首先国内这...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""楼主"",\n ""...","[B-PER-neutral, I-PER-neutral, O, O, O, O, O, ..."
4,matthiasxy,(从原英文留言翻译过来，这个社区不支持英文留言) 很抱歉打扰您！ 我不知道如何帮助您！ 或者...,2024-04-22,China_irl,en,"[\n {\n ""entity"": ""HR"",\n ""...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5,rioysl,鼓动你的 大多都躲在安全的地方闪煽动热血青年 最后出事也是热血青年 他们分毫不伤下次再忽悠一个继续,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""热血青年"",\n ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6,Particular-Fig4183,最近看到有润韩国庇护被拒的 可能会被遣返 恐怕只有接着润了 证据链这块最好咨询有法律背景或者...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""韩国"",\n ""...","[O, O, O, O, O, O, B-LOC-neutral, I-LOC-neutra..."
7,Particular-Fig4183,其实发了offer 顺利入职 开人都是要赔的 不论什么原因开人 除非是主动离职 实际操作上就...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""offer"",\n ...","[O, O, O, O, B-ORG-neutral, I-ORG-neutral, I-O..."
8,Alert_Yesterday9782,OP 不用太担心这件事情对自己日常生活的影响。我和你情况类似，时间更长，罪名更重（煽颠），甚...,2024-04-23,China_irl,zh,"[\n {\n ""entity"": ""OP"",\n ""...","[B-PER-neutral, I-PER-neutral, O, O, O, O, O, ..."
9,Particular-Fig4183,500强跨国大外企的背调真能查社保和有无案底 最严的会核查每个月的社保 看是否有出入 但是“...,2024-04-21,China_irl,zh,"[\n {\n ""entity"": ""500强跨国大外企"",\n ...","[B-ORG-neutral, I-ORG-neutral, I-ORG-neutral, ..."


In [126]:
df.tail(50)

,author,text,timestamp,subreddit,language,gpt_output,labels
15450,Lady0905,Han er kjent både for tidligere voldshendelser...,2024-05-19,norge,no,"[\n {\n ""entity"": ""Han"",\n ...","[B-PER-negative, O, O, O, O, O, O, O, O, O, O,..."
15451,redinina,La oss late som at det er alkoholen sin skyld ...,2024-05-19,norge,no,"[\n {\n ""entity"": ""han"",\n ...","[O, O, O, O, O, O, O, B-ORG-neutral, O, O, O, ..."
15452,CapnSlappin,En kjenning av politiet med tidligere straffba...,2024-05-19,norge,no,"[\n {\n ""entity"": ""politiet"",\n ...","[O, O, O, B-ORG-neutral, O, O, O, O, O, O, O, ..."
15453,LuxuryBeast,"Ah, den gode, gamle Byzantiske skikken!",2024-05-19,norge,no,"[\n {\n ""entity"": ""Byzantiske"",\n ...","[O, O, O, O, B-LOC-neutral, O]"
15454,Snorrep,Judo er stikkordet! Å få noen kontrollert ned ...,2024-05-19,norge,no,"[\n {\n ""entity"": ""Judo"",\n ...","[B-ORG-neutral, O, O, O, O, O, O, O, O, O, O, ..."
15455,Far_Sheepherder1825,"Saken er nok ikke avgjort enda, må vel innom d...",2024-05-18,norge,da,"[\n {\n ""entity"": ""Saken"",\n ...","[B-PER-neutral, O, O, O, O, O, O, O, O, B-ORG-..."
15456,TopptrentHamster,Fire uker er vel maksimalt hvor lenge man kan ...,2024-05-18,norge,no,"[\n {\n ""entity"": ""Fire uker"",\n ...","[O, O, O, O, O, O, O, O, O, O, O, O, B-OCC-neu..."
15457,overwatchher,Hvorfor skal festing være synonymt med slåsska...,2024-05-18,norge,da,"[\n {\n ""entity"": ""festing"",\n ...","[O, O, B-O-neutral, O, O, O, B-O-negative, O, O]"
15458,redinina,Kommer an på hvilke lovbrudd. Jeg synes ikke f...,2024-05-19,norge,no,"[\n {\n ""entity"": ""Kommer"",\n ...","[B-PER-neutral, O, O, O, O, O, O, O, B-ORG-neg..."
15459,wagieanonymous,"Kanskje det han blir dømt til, men hvor mye av...",2024-05-18,norge,no,"[\n {\n ""entity"": ""Kanskje"",\n ...","[B-O-neutral, O, B-PER-neutral, O, O, O, O, O,..."


In [127]:
some_row = df.loc[15480]
some_row

author                                        norway_is_awesome
text          Tiden i varetekt blir jo trukket fra endelig d...
timestamp                                            2024-05-19
subreddit                                                 norge
language                                                     da
gpt_output    [\n      {\n        "entity": "Tiden",\n      ...
labels              [B-TIME-neutral, O, O, O, O, O, O, O, O, O]
Name: 15480, dtype: object

In [128]:
print(len(some_row['labels']))

10


In [129]:
print(len(some_row['text']))

53


In [132]:
# Select the desired columns
df = df[["author", "timestamp", "subreddit", "language", "text", "labels"]]

# Remove rows where labels are an empty list
df = df[df['labels'].apply(lambda x: x != [])]

# Export to CSV
df.to_csv('/Users/vik/Desktop/cs224nproject/cleaned_data.csv', index=False, header=True)

